# VQSD Demo
This is a circuit demonstratinga (not yet) scalable implementation of the VQSD algorithm in pennylane.

In [34]:
# imports
import pennylane as qml
from pennylane import numpy as np

In [35]:
# define device
n_qubits = 1
dev = qml.device("default.qubit", wires=n_qubits*2)

In [36]:
n_layers = 2 # num parameter layers
shape = qml.RandomLayers.shape(n_layers=n_layers, n_rotations=2) # properly define the shape of the gate layers

In [37]:
# helper function for generating the operator for expval (since for some reason I can't find a way to do a @ b @ c @ ... in a compact fashion)
def generate_operator(n_qubits):
    if n_qubits == 0:
        return None
    if n_qubits == 1:
        return qml.PauliZ(0)
    return generate_operator(n_qubits - 1) @ qml.PauliZ(n_qubits - 1)

In [38]:
# define circuit
@qml.qnode(dev)
def circuit1(weights):
    for i in range(n_qubits*2): # initial state TODO: change this to be able to represent any initial matrix
        qml.Hadamard(i)
        # qml.RX(weights[0], wires=i)
        # qml.RY(weights[1], wires=i)
        # qml.RZ(weights[2], wires=i)
    qml.RandomLayers(weights=weights, wires=range(n_qubits)) # should be the alternating one but idk how to implement that
    qml.RandomLayers(weights=weights, wires=range(n_qubits, 2*n_qubits)) # there's a second circuit with the exact same weights as the first (cuz we have to calculate both the purity and the trace of the dephased matrix)
    
    # cost function (DIP test)
    qml.CNOT(wires=[0,1])
    operator = generate_operator(n_qubits)
    purity = qml.expval(qml.PauliZ(0)) # trace(p^2)
    dephase = qml.probs(wires=range(n_qubits, n_qubits*2)) # trace(dephased(p^2))
    return purity, dephase

print(qml.draw(circuit1, expansion_strategy="device")(weights))

0: ──H──RX(0.23)──RX(0.18)──RZ(0.06)──RY(0.39)─╭●──Z─┤  <Z>  
1: ──H──RX(0.23)──RX(0.18)──RZ(0.06)──RY(0.39)─╰X────┤  Probs


In [40]:
# cost function
def cost_fn(param):
    output = circuit1(param)
    return output[0] - output[1] # tr(p^2) - tr(Z(p^2))

In [39]:
#initialize random weights for the circuit
init_params = np.random.random(size=shape)
print(cost_fn(init_params))

-0.8379247465287891


In [41]:
# optimization loop
opt = qml.GradientDescentOptimizer(stepsize=0.4)
steps = 100
params = init_params

for i in range(steps):
    params = opt.step(cost_fn, params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost_fn(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5: -1.9999908
Cost after step    10: -2.0000000
Cost after step    15: -2.0000000
Cost after step    20: -2.0000000
Cost after step    25: -2.0000000
Cost after step    30: -2.0000000
Cost after step    35: -2.0000000
Cost after step    40: -2.0000000
Cost after step    45: -2.0000000
Cost after step    50: -2.0000000
Cost after step    55: -2.0000000
Cost after step    60: -2.0000000
Cost after step    65: -2.0000000
Cost after step    70: -2.0000000
Cost after step    75: -2.0000000
Cost after step    80: -2.0000000
Cost after step    85: -2.0000000
Cost after step    90: -2.0000000
Cost after step    95: -2.0000000
Cost after step   100: -2.0000000
Optimized rotation angles: [[ 2.12339111e-01  1.81824967e-01]
 [-8.27329658e-18  1.57079633e+00]]
